In [ ]:
import numpy as np
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm
import dlib
import random
import ipyplot

from utils import *
from bbox import *

In [ ]:
class GetModelFaceDescriptor(object):
    def __init__(self, face_recognition_model_v1_path):
        self.facerec = dlib.face_recognition_model_v1(face_recognition_model_v1_path)

    def crop_rotate_clip(self, model):
        image = Image.open(model.filename).convert("RGB")
        if_portrait = model.portrait * 90
        image = image.rotate(if_portrait, expand=1)
        faces = []
        for face in model.faces:
            face_clip = (
                image.rotate(face.rotation, expand=1)
                .crop((face.bbox.x1, face.bbox.y1, face.bbox.x2, face.bbox.y2))
                .resize(size=(150, 150), resample=Image.Resampling.LANCZOS)
            )
            faces.append(face_clip)
        return np.stack(faces)

    def get_vectors(self, model):
        faces = self.crop_rotate_clip(model)
        faces_vectors = np.array(self.facerec.compute_face_descriptor(faces)).tolist()
        for i, face in enumerate(model.faces):
            face.face_vector = faces_vectors[i]
        return model

In [ ]:
load_dotenv()
Migrator().run()
query = CONTAINER_IMAGE_FOLDER.split("/")[1]
models = Model.find(Model.filename % query).all()
models = [model for model in models if model.faces]
max_batch_size = 32

In [ ]:
rmi = RedisModelIterator(models, max_batch_size)

```
curl "https://raw.githubusercontent.com/davisking/dlib-models/master/dlib_face_recognition_resnet_model_v1.dat.bz2" \
    --compressed \
    --output "dlib_face_recognition_resnet_model_v1.dat.bz2" \
    && bzip2 -dk dlib_face_recognition_resnet_model_v1.dat.bz2
```

In [ ]:
face_recognition_model_v1_path = "dlib_face_recognition_resnet_model_v1.dat"
get_model_face_descriptor = GetModelFaceDescriptor(face_recognition_model_v1_path)
get_model_face_descriptor.get_vectors(models[0])

In [ ]:
est_total = (len(models) // max_batch_size) + 1
pbar = tqdm(total=est_total, desc="Calculating face vectors.")

face_clip_results = []
faceclip_exceptions = []
with ThreadPoolExecutor(max_workers=max_batch_size) as executor:
    for model_batch in rmi:
        futures = []
        for model in model_batch:
            futures.append(
                executor.submit(get_model_face_descriptor.get_vectors, model=model)
            )

        for future in as_completed(futures):
            try:
                face_clip_results.append(future.result())
            except BaseException as e:
                faceclip_exceptions.append({e: future})

        pbar.update()

In [ ]:
for model in face_clip_results:
    model.save()

In [ ]:
face_vectors = []
for model in models:
    for face in model.faces:
        if face.face_vector:
            face_vectors.append(dlib.vector(face.face_vector))


# face_vectors = np.stack(face_vectors)

labels = dlib.chinese_whispers_clustering(face_vectors, 0.5)

In [ ]:
i = 0
for model in models:
    for face in model.faces:
        if face.face_vector:
            face.label = labels[i]
            i += 1
    model.save()

In [ ]:
query = CONTAINER_IMAGE_FOLDER.split("/")[1]
models = Model.find(Model.filename % query).all()
models = [model for model in models if model.faces]

random.shuffle(models)

model = models[0]

labels = [face.label for face in model.faces]
crops = crop_and_rotate_clip(model, 1, 1)

ipyplot.plot_images(
    crops,
    labels=labels,
    show_url=False,
)